In [4]:
import math

In [84]:
# Incremento de variable
def incremento(vrbl, incrmnt):
    decimal = vrbl%1
    entero = vrbl//1

    #Generic (0.05, 0.1, 0.5)
    if ( (1/incrmnt)%2 == 0 ):
        vrbl += incrmnt

    # Symmetric by 0.5 (0.2)
    elif ( (1/incrmnt)%1 == 0 ):
        if (0 <= decimal < 0.5) & (decimal + incrmnt > 0.5):
            vrbl = entero + 0.5
        elif (decimal == 0.5):
            vrbl += incrmnt/2
        else:
            vrbl += incrmnt
            
    # error output (0.15, 0.3)         
    else:
        print('no itera')
    
    return vrbl   

In [85]:
def troncopiramidal(area_inf, area_sup, altura):
    volumen = altura / 3 * (area_inf + area_sup + math.sqrt(area_inf * area_sup))
    return volumen

In [86]:
def round_up(number, decimals):
    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

def round_down(number, decimals):
    r_r = round_up(number, decimals) - 10**-decimals
    r_r_plus_add = r_r + 10**-decimals
    return r_r_plus_add if (r_r_plus_add == number) else r_r

In [87]:
def mround_up(x, multiple):
    return math.ceil(x / multiple) * multiple

def mround_down(x, multiple):
    r_r = mround_up(x, multiple) - multiple
    return r_r + multiple if math.isclose(x%multiple, multiple) else r_r

In [89]:
def proporcion(X,Y, y):
    # X/Y == x/y
    return y * X/Y